# Getting started with Echopype

This notebook shows the minimum steps to start using Echopype.
We will focus on the steps to convert instrument-generated raw data files, compute the volume backscattering strength ($\mathrm{S_V}$) based on calibration and environmental parameters already stored in the file, and plot an echogram.
We will use a `.raw` file generated by a Simrad EK80 echosounder as an example.
Please see the documentation for usage with other supported formats.

In [1]:
import warnings

import echopype as ep  # we recommend using "ep"
import xarray as xr
import hvplot.xarray  # quick interactive plots!

import matplotlib.pyplot as plt

Here we will use an example file collected as part of the [Hake survey](https://www.fisheries.noaa.gov/west-coast/science-data/joint-us-canada-integrated-ecosystem-and-pacific-hake-acoustic-trawl-survey) run by the NOAA Northwest Fisheries Science Center, stored in an [open AWS data register from NOAA NCEI](https://registry.opendata.aws/ncei-wcsd-archive/).

Here we directly access and convert data from an S3 bucket without having to download them first. Echopype supports accessing data from multiple sources, such as a local filesystem (e.g., hard disk), a cloud object store (e.g., AWS S3), an http server, etc.

## Convert raw data

In [2]:
raw_path = "s3://noaa-wcsd-pds/data/raw/Bell_M._Shimada/SH2306/EK80/Hake-D20230804-T171641.raw"
ed = ep.open_raw(
    raw_path,
    sonar_model="EK80",
    storage_options={"anon": True},  # open bucket, otherwise need credential
)

In [3]:
type(ed)

echopype.echodata.echodata.EchoData

Let's take a look at what's in `ed`, and `EchoData` object.

An `EchoData` object is a representation of an echopype-adapted version of the SONAR-netCDF4 version 1 convention. It shows the tree-like group structure in the netCDF data model. We aim for the conversion from raw data to the `EchoData` object as lossless as possible.

To see what we changed from the convention and why, please see the **Data standardization** section of the [Echopype paper](https://doi.org/10.1093/icesjms/fsae133).

There are a few things to note in this example:
- All backscatter data are in the `Sonar/Beam_group1` group
- The backscatter data and many other variables are aligned in 3 major dimensions: `channel`, `ping_time`, `range_sample`
    - `channel` includes all the transducer channel, and for EK80 files these are the channel IDs configured in the EK80 system during data collection.
    - We chose to use `channel` instead of frequency to align the data, because some setups use multiple transducers transmitting at the same frequency but pointing in different directions. Additionally, "frequency" can be a misnomer broadband transmissions
    - We store the "frequency" of the channels in the variable `nominal_frequency` for easy access
    - `range_sample` are 0-based sample count of the received echoes
- Useful overview type of information are stored in the `Top-level` group, which is the "root" of the netCDF file
- Raw NMEA datagrams are found in the `Platform/NMEA` group
- Raw CONFIG datagram, which is an XML string, is save in the variable `content` in the `Vendor_specific` group

In addition, since `ed` is just converted and has not been saved, everything is stored **in-memory**, as indicated in the first line printed out below.

In [4]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data, including split-beam angle data when they exist.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

Echopype supports saving the `EchoData` object into a netCDF file (`.nc`) or a zarr store (`.zarr`). The groups and the representation stay the same, just the format specification is different. NetCDF files are single files that are easy to handle on a conventional filesystem. Zarr stores are optimized for cloud storage, but can be messy to access on a local filesystem as it would show up as many many folder. Both formats and the tools to access them continue to develop, and in particular there are recent new packages that allow representing multiple netCDF files as a virtual zarr store, so things are becoming more convenient and powerful on the user side!

In [5]:
# Save to a zarr file
export_path = "./exports/getting_started/ed_example.zarr"

ed.to_zarr(export_path, overwrite="w")

We can load the saved zarr back using the `open_converted` function.

By using `chunks={}`, we "lazy-load" all the data in this zarr file into a new `EchoData` object, `ed_new`, which can be seen as a representation or a handle of the data stored on disk.

This capability is very important for distributed computing and is key to out-of-core computation, because different "workers" can directly access different parts of the same file on disk, and don't have load the entire dataset into memory. This file is pretty small, but in the `OOI_eclipse.ipynb` notebook, you can see an example in which the zarr file is approximately 8 GB and its computation would generally overwhelm a regular laptop without distributed computing capability.

In [6]:
ed_new = ep.open_converted(export_path, chunks={})
ed_new

<EchoData: standardized raw data from /Users/wujung/code_git/echopype-examples/notebooks/exports/getting_started/ed_example.zarr>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data, including split-beam angle data when they exist.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

We can find out if a variable is lazy-loaded or completely loaded into memory by looking at the representation:

In [7]:
# In-memory: see all the values
ed["Sonar/Beam_group1"]["backscatter_r"]

<xarray.DataArray 'backscatter_r' (channel: 5, ping_time: 214,
                                   range_sample: 36198)> Size: 155MB
array([[[  10.594845  ,   14.087263  ,   16.239157  , ...,
                   nan,           nan,           nan],
        [  10.75947   ,   14.240129  ,   16.380264  , ...,
                   nan,           nan,           nan],
        [  10.688916  ,   14.169576  ,   16.33323   , ...,
                   nan,           nan,           nan],
        ...,
        [  10.700676  ,   14.193093  ,   16.344988  , ...,
                   nan,           nan,           nan],
        [  10.665399  ,   14.157817  ,   16.32147   , ...,
                   nan,           nan,           nan],
        [  10.618362  ,   14.110781  ,   16.274433  , ...,
                   nan,           nan,           nan]],

       [[ -11.641395  ,   -5.6090355 ,   -0.42332342, ...,
         -121.61141   , -120.52959   , -119.64766   ],
        [ -11.723707  ,   -5.6090355 ,   -0.4350824 , ...,
         -122.08177   , -123.09305   , -124.833374  ],
        [ -11.676671  ,   -5.6090355 ,   -0.4350824 , ...,
         -132.58255   , -135.21655   , -134.75795   ],
...
        [   3.5159361 ,   10.406701  ,   15.780557  , ...,
                   nan,           nan,           nan],
        [   3.4336233 ,   10.312629  ,   15.651208  , ...,
                   nan,           nan,           nan],
        [   3.6452851 ,   10.512531  ,   15.85111   , ...,
                   nan,           nan,           nan]],

       [[   6.655585  ,    9.465982  ,   11.359179  , ...,
                   nan,           nan,           nan],
        [   6.726139  ,    9.548295  ,   11.441491  , ...,
                   nan,           nan,           nan],
        [   6.8084517 ,    9.630608  ,   11.523805  , ...,
                   nan,           nan,           nan],
        ...,
        [   6.820211  ,    9.642367  ,   11.523805  , ...,
                   nan,           nan,           nan],
        [   6.8319697 ,    9.654126  ,   11.57084   , ...,
                   nan,           nan,           nan],
        [   6.8319697 ,    9.654126  ,   11.547322  , ...,
                   nan,           nan,           nan]]], dtype=float32)
Coordinates:
  * channel       (channel) <U25 500B 'WBT 400140-15 ES120-7C_ES' ... 'WBT 40...
  * ping_time     (ping_time) datetime64[ns] 2kB 2023-08-04T17:16:41.811981 ....
  * range_sample  (range_sample) int64 290kB 0 1 2 3 ... 36194 36195 36196 36197
Attributes:
    long_name:  Raw backscatter measurements (real part)
    units:      dB

In [8]:
# Lazy-loaded: see the entire array size and the chunk size
ed_new["Sonar/Beam_group1"]["backscatter_r"]

<xarray.DataArray 'backscatter_r' (channel: 5, ping_time: 214,
                                   range_sample: 36198)> Size: 155MB
dask.array<open_dataset-backscatter_r, shape=(5, 214, 36198), dtype=float32, chunksize=(5, 214, 23364), chunktype=numpy.ndarray>
Coordinates:
  * channel       (channel) <U25 500B 'WBT 400140-15 ES120-7C_ES' ... 'WBT 40...
  * ping_time     (ping_time) datetime64[ns] 2kB 2023-08-04T17:16:41.811981 ....
  * range_sample  (range_sample) int64 290kB 0 1 2 3 ... 36194 36195 36196 36197
Attributes:
    long_name:  Raw backscatter measurements (real part)
    units:      dB

Note that the zarr or netCDF files on disk are actually standard files, and can be access using regular xarray functions. For example, we an access the GPS data stored in the `Platform` group directly using `xarray.open_dataset`:

In [9]:
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=RuntimeWarning)  # ignore warning to focus on tutorial message

    ds_platform = xr.open_dataset(f"{export_path}/Platform", engine="zarr")

In [10]:
ds_platform

<xarray.Dataset> Size: 69kB
Dimensions:                      (channel: 5, time2: 989, time1: 784, time3: 1)
Coordinates:
  * channel                      (channel) <U25 500B 'WBT 400140-15 ES120-7C_...
  * time1                        (time1) datetime64[ns] 6kB 2023-08-04T17:16:...
  * time2                        (time2) datetime64[ns] 8kB 2023-08-04T17:16:...
  * time3                        (time3) datetime64[ns] 8B 2023-08-04T17:16:4...
Data variables: (12/26)
    MRU_offset_x                 float64 8B ...
    MRU_offset_y                 float64 8B ...
    MRU_offset_z                 float64 8B ...
    MRU_rotation_x               float64 8B ...
    MRU_rotation_y               float64 8B ...
    MRU_rotation_z               float64 8B ...
    ...                           ...
    transducer_offset_x          (channel) float64 40B ...
    transducer_offset_y          (channel) float64 40B ...
    transducer_offset_z          (channel) float64 40B ...
    vertical_offset              (time2) float64 8kB ...
    water_level                  float64 8B ...
    water_level_draft_is_manual  int64 8B ...
Attributes:
    platform_code_ICES:  
    platform_name:       
    platform_type:

## Compute Sv

We can then proceed to compute Sv from the raw data records. Here, we will use the calibration and environmental parameters already stored in the original instrument-generated `.raw` file to do it. Echopype also supports updating these parameters as part of the function call, or use an Echoview `.ECS` file (though note that this functionality is in beta and may not cover all the equivalent vocabulary recognized by Echoview).

`compute_Sv` requires two arguments: `waveform_mode` and `encode_mode`, because EK80 files can contain data from a variety of sampling configurations, including:
- narrowband data stored in power-angle format (`waveform_mode="CW", encode_mode="power"`)
- narrowband data stored in complex format (`waveform_mode="CW", encode_mode="complex"`)
- broadband data stored in complex format (`waveform_mode="FM", encode_mode="complex"`)

For AZFP echosounder, the arguments will always be `waveform_mode="CW", encode_mode="power"` since the instrument does not yet support broadband transmissions.

In [13]:
# Try to compute Sv without argument will error out
ds_Sv = ep.calibrate.compute_Sv(ed)

ValueError: waveform_mode and encode_mode must be specified for EK80 calibration

:::{note}
Echopype does not yet support ping-sequence files! We expect that to be part of an upcoming releases.
:::

With the correct argument combination is used, we can proceed to take a look at the Sv output:

In [17]:
# Use the correct combination
ds_Sv = ep.calibrate.compute_Sv(ed, waveform_mode="CW", encode_mode="power")
ds_Sv

<xarray.Dataset> Size: 620MB
Dimensions:                        (channel: 5, ping_time: 214,
                                    range_sample: 36198, filenames: 1)
Coordinates:
  * channel                        (channel) <U25 500B 'WBT 400140-15 ES120-7...
  * ping_time                      (ping_time) datetime64[ns] 2kB 2023-08-04T...
  * range_sample                   (range_sample) int64 290kB 0 1 ... 36197
  * filenames                      (filenames) int64 8B 0
Data variables: (12/24)
    Sv                             (channel, ping_time, range_sample) float64 310MB ...
    echo_range                     (channel, ping_time, range_sample) float64 310MB ...
    frequency_nominal              (channel) float64 40B 1.2e+05 ... 2e+05
    sound_speed                    float64 8B 1.48e+03
    sound_absorption               (channel) float64 40B 0.03281 ... 0.0463
    temperature                    float64 8B 6.9
    ...                             ...
    beamwidth_athwartship          (channel) float64 40B 6.74 10.6 ... 6.81 6.07
    impedance_transducer           (channel) float64 40B 75.0 75.0 ... 75.0 75.0
    impedance_transceiver          (channel) int64 40B 5400 5400 5400 5400 5400
    receiver_sampling_frequency    (channel) float64 40B 1.5e+06 ... 1.5e+06
    source_filenames               (filenames) <U82 328B 's3://noaa-wcsd-pds/...
    water_level                    float64 8B 0.0
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.10.1
    processing_time:              2025-04-06T02:27:54Z
    processing_function:          calibrate.compute_Sv

Different from the `EchoData` object, which is a dataset with a hierarchical group structure, `ds_Sv` is a standard xarray `Dataset` with multiple data variable, each an xarray `DataArray`.

In addition to `Sv`, there are many other variables in this dataset, including:
- all calibration and environmental parameters used to compute the Sv values, to ensure traceability and reproducibility
- the `echo_range` variable that contains the range of each echo sample from the transducer

`echo_range` has the same dimension as the `Sv` variable, because the sampling rate of the echo time series (step size along range) and sound speed may vary across pings depending on the echosounder configuration and the environment.

In [ ]:
from echopype.qc import exist_reversed_time, coerce_increasing_time
if exist_reversed_time(ds_Sv, "ping_time"):
    # Coerce increasing time
    coerce_increasing_time(ds_Sv)

### Add additional information to the Sv dataset

The Sv dataset generated by `compute_Sv` only contains backscatter data, but we often need additional information, such as the latitude/longitutude location of the platform, or the split-beam angle inferred from the echo time series. We can add these into the dataset by calling functions from the `consolidate` subpackage. We can also compute the depth of each echo sample if the platform depth is known.

:::{note}
Echopype does not yet support heave compensation.
:::

In [23]:
# Add depth
# depth_offset=9.8 because transducer was on a lowered centerboard at 9.8 m deep
ds_Sv = ep.consolidate.add_depth(ds_Sv, ed, depth_offset=9.8)

In [53]:
# Add split-beam angle
ds_Sv = ep.consolidate.add_splitbeam_angle(ds_Sv, ed, waveform_mode="CW", encode_mode="power", to_disk=False)

In [48]:
# Add geographical location
ds_Sv = ep.consolidate.add_location(ds_Sv, ed, nmea_sentence="GGA")

ValueError: The ``echodata["Platform"]["time1"]`` array contains duplicate values. Downstream interpolation on the position variables requires unique time values.

Here, we see that the 

In [50]:
ed["Platform"]

<xarray.Dataset> Size: 69kB
Dimensions:                      (time1: 784, time2: 989, channel: 5, time3: 1)
Coordinates:
  * channel                      (channel) <U25 500B 'WBT 400140-15 ES120-7C_...
  * time1                        (time1) datetime64[ns] 6kB 2023-08-04T17:16:...
  * time2                        (time2) datetime64[ns] 8kB 2023-08-04T17:16:...
  * time3                        (time3) datetime64[ns] 8B 2023-08-04T17:16:4...
Data variables: (12/26)
    latitude                     (time1) float64 6kB 43.28 43.28 ... 43.28 43.28
    longitude                    (time1) float64 6kB -124.9 -124.9 ... -124.9
    sentence_type                (time1) <U3 9kB 'GGA' 'GLL' ... 'GLL' 'GLL'
    pitch                        (time2) float64 8kB 0.79 0.66 ... 1.05 1.03
    roll                         (time2) float64 8kB -0.5 -0.5 ... -1.16 -1.19
    vertical_offset              (time2) float64 8kB -0.03 0.01 ... -0.31 -0.33
    ...                           ...
    position_offset_y            float64 8B nan
    position_offset_z            float64 8B nan
    frequency_nominal            (channel) float64 40B 1.2e+05 1.8e+04 ... 2e+05
    heading                      (time2) float64 8kB 276.4 276.4 ... 275.6 275.6
    latitude_mru1                (time3) float64 8B nan
    longitude_mru1               (time3) float64 8B nan
Attributes:
    platform_name:       
    platform_type:       
    platform_code_ICES:

In [59]:
ed["Platform"] = ed["Platform"].where((ed["Platform"]["sentence_type"]=="GGA"), drop=True)

In [ ]:
ed["Platform"]["time1"].sel().diff("time1")

<xarray.DataArray 'time1' (time1: 783)> Size: 6kB
array([         0,          0,  822365000,  181248000,   79187000,
        697380000,  181246000,  128748000,  744841000,  181247000,
         64198000,  745874000,  181246000,   79189000,  697382000,
        181246000,  112722000,  745873000,  181248000,   79189000,
        745874000,  181248000,   79187000,  783551000,  207800000,
        179676000,  566223000,  227079000,   33337000,  737717000,
        253629000,  177635000,  621870000,  206803000,  163717000,
        609446000,  383329000, -149183000,  735678000,  254674000,
        179675000,  573999000,  414314000, -161678000,  751635000,
        238718000,  180673000,  573998000,  414314000, -161678000,
        751634000,  238718000,  180672000,  826637000, -202236000,
        383328000,  570541000,  254677000,  164713000,  574001000,
        251638000,  178631000,  622870000,  205806000,  164713000,
        825640000, -217180000,  383327000,  586483000,  254677000,
        164715000,  825638000, -217196000,  383330000,  549820000,
        181247000,  109110000,  753628000,  253679000,  164725000,
        825628000, -216198000,  383325000,  548827000,  181245000,
        109110000,  753628000,  237723000,  181669000,  573999000,
        414314000, -163673000,  753630000,  252680000,  166708000,
        823864000, -215397000,  383325000,  586479000,  252682000,
...
         87171000,  738918000,  181246000,   87139000,  775570000,
        230742000,  155737000,  567201000,  227080000,   40332000,
        738924000,  227080000,   41305000,  778607000,  211746000,
        218525000,  573999000,  214783000,  172693000,  566201000,
        227079000,   40337000,  722934000,  227082000,   40336000,
        794668000,  212639000,  202568000,  520367000,  181248000,
         86170000,  738893000,  181248000,   86170000,  777563000,
        213788000,  172691000,  566200000,  227081000,   40337000,
        739889000,  227081000,   24378000,  794565000,  211869000,
        218405000,  537321000,  181245000,   69216000,  739891000,
        181248000,   69214000,  740888000,  181248000,   85171000,
        793524000,  196830000,  173689000,  567198000,  227081000,
         39339000,  739890000,  227079000,   39360000,  687403000,
        227081000,   75853000,  764466000,  227079000,   15759000,
        740888000,  227080000,   22384000,  740887000,  227081000,
         39339000,  740889000,  227079000,   38342000,  740888000,
        227080000,   22386000,  740965000,  227080000,   38265000,
        741884000,  227081000,   22385000,  740913000,  227082000,
         38315000,  741884000,  227079000,   37346000,  681688000,
        309672000,  173635000,  574002000], dtype='timedelta64[ns]')
Coordinates:
  * time1    (time1) datetime64[ns] 6kB 2023-08-04T17:16:41.811981 ... 2023-0...

In [9]:
del ed1

In [59]:
ed = ep.open_raw(
    "s3://" + s3rawfiles[0],
    sonar_model="EK80",
    storage_options={"anon": True},  # open bucket, otherwise need credential
)